In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE

# Load the dataset
df = pd.read_csv('Train_Test_IoT_Garage_Door.csv')

#Data Cleaning

# Ensure 'sphone_signal' is numeric by converting and handling non-numeric entries
df['sphone_signal'] = pd.to_numeric(df['sphone_signal'], errors='coerce')  # Convert non-numeric to NaN

# Handling missing values (mean for numerical, mode for categorical)
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Impute missing values for numerical columns
df[['sphone_signal']] = num_imputer.fit_transform(df[['sphone_signal']])

# Impute missing values for categorical columns (if needed)
df[['door_state']] = cat_imputer.fit_transform(df[['door_state']])

#  Handling Outliers using Isolation Forest (for numerical columns)
iso = IsolationForest(contamination=0.01)  # Tune contamination rate if needed
outliers = iso.fit_predict(df[['sphone_signal']])

# Remove rows identified as outliers (-1 label from Isolation Forest)
df = df[outliers != -1]

#  One-Hot Encoding for categorical columns (door_state)
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap
door_state_encoded = pd.DataFrame(encoder.fit_transform(df[['door_state']]), 
                                  columns=encoder.get_feature_names_out(['door_state']))

# Concatenate the encoded columns back to the main dataframe and drop original 'door_state'
df = pd.concat([df, door_state_encoded], axis=1)
df.drop('door_state', axis=1, inplace=True)

#  Convert 'type' column to numeric labels if needed (using LabelEncoder)
label_encoder = LabelEncoder()
df['type_encoded'] = label_encoder.fit_transform(df['type'])

# Drop the original 'type' column if you're using the encoded version
df.drop('type', axis=1, inplace=True)

#  Standardise numerical columns (sphone_signal)
scaler = StandardScaler()
df[['sphone_signal']] = scaler.fit_transform(df[['sphone_signal']])

# Drop non-numeric columns such as 'date' and 'time' before applying SMOTE
df.drop(['date', 'time'], axis=1, inplace=True)

#  Handling Imbalanced Data using SMOTE
smote = SMOTE(sampling_strategy='auto')  # 'auto' balances all classes
X = df.drop(['label'], axis=1)  # All features except the label
y = df['label']  # Target labels

# Apply SMOTE to balance the classes
X_smote, y_smote = smote.fit_resample(X, y)

# Combine back the balanced dataset
df_balanced = pd.concat([X_smote, y_smote], axis=1)

# Save the preprocessed dataset
df_balanced.to_csv('Preprocessed_Garage_Door_Dataset.csv', index=False)

print("Preprocessing complete. Dataset saved as 'Preprocessed_Garage_Door_Dataset.csv'.")


C:\Users\krmic\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Preprocessing complete. Dataset saved as 'Preprocessed_Garage_Door_Dataset.csv'.
